# Interfaces, Protocols, and ABCs

Ways to define and use interfaces in python
1. Duck typing
2. Goose typing
3. Static typing
4. Static duck typing

## Two Kinds of Protocols

Network protocol - specifies commands that a client can send to a server, e.g. `GET, PUT, HEAD`.

Object protocol - specifies methods which an object must provide to fulfill a role.

Example 13-1. Partial sequence protocol implementation with `__getitem__`

In [1]:
import numbers


class Vowels:
    def __getitem__(self, i):
        return "AEIOU"[i]

v = Vowels()
v[0]

'A'

In [2]:
v[-1]

'U'

In [3]:
for c in v:
    print(c)

A
E
I
O
U


In [4]:
"E" in v

True

In [5]:
"Z" in v

False

Implementing `__getitem__` is enough to allow retrieving items by index, iteration with `for`, and membership checks
with `in`. We'll call this a _Dynamic protocol_.

There is also protocol that we saw before - subclasses of `typing.Protocol` to define one or more methods that a
class must implement (or inherit) to satisfy a type checker.
We'll call this a _Static protocol_.

Differences between static and dynamic protocols:
- An object may implement part of a dynamic protocol and still be useful. A static protocol would require every
method declared in teh protocol class to be implemented, even if we don't need them.
- Static protocols can be verified by static type checkers. Dynamic protocols can't.

In addition to static protocols, python defines another way of defining an explicit interface in the code: an ABC.

## Programming Ducks

Recall the `Vowels` class. It does not inherit from `abc.Sequence`, and it o nly implements `__getitem__`.

Why is it iterable?

In [6]:
len(v)

TypeError: object of type 'Vowels' has no len()

Example 13-2. A deck as a sequence of cards (same as example 1-1)

In [7]:
import collections

Card = collections.namedtuple("Card", ["rank", "suit"])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


## Monkey Patching: Implementing a Protocol at Runtime

The `FrenchDeck` is missing an essential feature: it cannot be shuffled.

In [8]:
from random import shuffle

l = list(range(10))
shuffle(l)
l

[1, 7, 8, 0, 6, 5, 3, 2, 9, 4]

Example 13-3. `random.shuffle` cannot handle `FrenchDeck`

In [9]:
from random import shuffle

deck = FrenchDeck()

shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

Example 13-4. Monkey patching `FrenchDeck` to make it mutable and compatible with `random.shuffle`

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)

In [11]:
deck[:5]

[Card(rank='10', suit='hearts'),
 Card(rank='5', suit='diamonds'),
 Card(rank='9', suit='clubs'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='K', suit='hearts')]

## Defensive Programming and "Fail Fast"

Example 13-5. Duck typing to handle a string or an iterable of strings


In [12]:
try:
    field_names = field_names.replace(",", " ").split()
except AttributeError:
    pass

field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise ValueError("field_names must all be valid identifiers")

NameError: name 'field_names' is not defined

Duck typing can be more expressive than static type hints. There is no way to spell a type hint that says
"`field_names` must be a string of identifiers separated by spaces or commas."

## Goose Typing

What _goose typing_ means is: `isinstance(obj, cls)` is now just fine... as long as `cls` is an abstract base class.

In [13]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

### Subclassing an ABC

Example 13-6. `frenchdeck2.py`: `FrenchDeck2`, a subclass of `collections.MutableSequence`

In [14]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)


In [15]:
f = FrenchDeck2()


### ABCs in the Standard Library

Example 13-7. `tombola.py`: `Tombola` is an `ABC` with two abstract methods and two concrete methods.

In [16]:
# tag::TOMBOLA_ABC[]

import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """
        print('inside abstract pick')

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(items)


# end::TOMBOLA_ABC[]


In [17]:
class MyTombola(Tombola):
    def pick(self):
        super(MyTombola, self).pick()  # Jamie to look into this
        print("inside MyTombola pick")

    def load(self, iterable):
        pass

MyTombola().pick()

inside abstract pick
inside MyTombola pick


Example 13-8. A fake `Tombola` doesn't go undetected

In [18]:
class Fake(Tombola):
    def pick(self):
        return 13

In [19]:
Fake

__main__.Fake

In [20]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

## Subclassing an ABC

Example 13-9. `bingo.py`: `BingoCage` is a concrete subclass of `Tombola`

In [21]:
# tag::TOMBOLA_BINGO[]

import random

from tombola import Tombola


class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <6>
        self.pick()

# end::TOMBOLA_BINGO[]


In [22]:
b = BingoCage("abc")

In [23]:
b.pick()

'c'

In [24]:
b.pick()

'b'

In [25]:
b.pick()

'a'

In [26]:
b.pick()

LookupError: pick from empty BingoCage

In [27]:
b()

LookupError: pick from empty BingoCage

Example 13-10. `lotto.py`: `LottoBlower` is a concrete subclass that overrides the `inspect` and `loaded` methods
from `Tombola`

In [28]:
# tag::LOTTERY_BLOWER[]

import random

from tombola import Tombola


class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # <1>

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  # <3>

    def loaded(self):  # <4>
        return bool(self._balls)

    def inspect(self):  # <5>
        return tuple(self._balls)


# end::LOTTERY_BLOWER[]


## A Virtual Subclass of an ABC

Example 13-11. `tombolist.py`: class `TomboList` is a virtual subclass of `Tombola`


In [29]:
from random import randrange

from tombola import Tombola

@Tombola.register  # <1>
class TomboList(list):  # <2>

    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5>

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(self)

# Tombola.register(TomboList)  # <7>

In [30]:
issubclass(TomboList, Tombola)

True

In [31]:
t = TomboList(range(100))

In [32]:
isinstance(t, Tombola)

True

In [33]:
TomboList.__mro__

(__main__.TomboList, list, object)

`Tombola` is not in `Tombolist.__mro__`, so `Tombolist` does not inherit any methods from `Tombola`.

## Structural Typing with ABCs

In [34]:
class Struggle:
    def __len__(self):
        return 23


In [35]:
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [36]:
issubclass(Struggle, abc.Sized)

True

Example 13-12. Definition of `Sized` from the source code of Lib/_collections_abc.py

In [37]:
from abc import (
    ABCMeta,
    abstractmethod,
)

class Sized(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True

        return NotImplemented

Sized.__subclasshook__(some_object)

subclasshook(Sized, some_object)

subclasshook(Iterable, some_object)

NameError: name 'some_object' is not defined

## Static Protocols

### The Typed double Function

In [38]:
"""
A static protocol makes it possible to annotate and type check the `double()` function from previous chapter.
"""

def double(x):
    return x * 2

In [39]:
double(1.5)

3.0

In [40]:
double("A")

'AA'

In [41]:
double([10, 20, 30])

[10, 20, 30, 10, 20, 30]

In [42]:
from fractions import Fraction
double(Fraction(2, 5))

Fraction(4, 5)

Before static protocols were introduced, there was no practical way to add type hints to `double` without limiting
its possible uses.

Example 13-13. `double_protocol.py`: definition of `double` using a protocol

In [43]:
from typing import TypeVar, Protocol

T = TypeVar('T')  # <1>

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...  # <2>

RT = TypeVar('RT', bound=Repeatable)  # <3>

def double(x: RT) -> RT:  # <4>
    return x * 2


In [44]:
{} * {}

TypeError: unsupported operand type(s) for *: 'dict' and 'dict'

In [45]:
double({})

TypeError: unsupported operand type(s) for *: 'dict' and 'int'

## Runtime Checkable Static Protocols

Example 13-14. `typing.SupportsComplex` protocol source code

In [46]:
from typing import runtime_checkable
from abc import abstractmethod


@runtime_checkable
class SupportsComplex(Protocol):
    """An ABC with one abstract method __complex__"""
    __slots__ = ()

    @abstractmethod
    def __complex__(self) -> complex:
        pass

In [47]:
complex(3)

(3+0j)

Example 13-15. Using `SupportsComplex` at runtime

In [48]:
from typing import SupportsComplex
import numpy as np

In [49]:
c64 = np.complex64(3+4j)

In [50]:
isinstance(c64, complex)

False

In [51]:
isinstance(c64, SupportsComplex)

True

In [52]:
c = complex(c64)

In [53]:
c

(3+4j)

In [54]:
isinstance(c, SupportsComplex)

False

In [55]:
complex(c)

(3+4j)

In [56]:
isinstance(c, (complex, SupportsComplex))

True

In [57]:
import numbers

isinstance(c, numbers.Complex)

True

In [58]:
isinstance(c64, numbers.Complex)

True

## Duck typing is your friend

Instead of calling `isinstance` or `hasattr`, just try the operations you need to do on the object and handle
exceptions as needed.

In [59]:
o = object()
if isinstance(o, (complex, SupportsComplex)):
    # do something that requires `o` to be convertible to complex
    pass
else:
    raise TypeError("o must be convertible to complex")

TypeError: o must be convertible to complex

In [60]:
# The goose typing approach
import numbers

if isinstance(o, numbers.Complex):
    # do something with `o`, an instance of `Complex`
    pass
else:
    raise TypeError("o must be an instance of Complex")

TypeError: o must be an instance of Complex

In [61]:
# Many pythonistas will live by EAFP: it's easier to ask for forgiveness than permission

try:
    c = complex(o)
except TypeError as exc:
    raise TypeVar("o must be convertible to complex") from exc

TypeError: exceptions must derive from BaseException

In [62]:
# And, if all you're going to do is raise a `TypeError` anyway, then why bother with the try / except?

c = complex(o)

TypeError: complex() first argument must be a string or a number, not 'object'

## Limitations of Runtime Protocol Checks

In [63]:
import sys

sys.version

'3.10.0 (default, Nov 10 2021, 11:24:47) [Clang 12.0.0 ]'

In [64]:
c = 3 + 4j

In [65]:
c.__float__

AttributeError: 'complex' object has no attribute '__float__'

In [66]:
c.__float__()

AttributeError: 'complex' object has no attribute '__float__'

In [67]:
# See the book for how this example worked on python 3.9! They changed in our version, 3.10

## Supporting a Static Protocol

Example 13-16. `vector2d_v4.py`: methods for converting to and from `complex`

In [68]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V4_COMPLEX[]
    def __complex__(self):
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum):
        return cls(datum.real, datum.imag)  # <1>

# end::VECTOR2D_V4_COMPLEX[]

In [69]:
complex(Vector2d(x=1 ,y=2))

(1+2j)

In [70]:
Vector2d.fromcomplex(complex(1, 2))

Vector2d(1.0, 2.0)

In [71]:
class CoolVector2d(Vector2d):

    def cool_vector(self):
        print('I am cool')

In [72]:
CoolVector2d.fromcomplex(complex(1, 2)).cool_vector()

I am cool


Given the preceding code and the `__abs__` method the `Vector2d` already had, we get these features:

In [73]:
from typing import SupportsComplex, SupportsAbs

v = Vector2d(3, 4)

In [74]:
isinstance(v, SupportsComplex)

True

In [75]:
isinstance(v, SupportsAbs)

True

In [76]:
complex(v)

(3+4j)

In [77]:
abs(v)

5.0

In [78]:
Vector2d.fromcomplex(3+4j)

Vector2d(3.0, 4.0)

Example 13-17. `vector2d_v5.py`: adding annotations to the methods under study

In [79]:
from typing import Iterator

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self) -> float:
        return self.__x

    @property
    def y(self) -> float:
        return self.__y

    def __iter__(self) -> Iterator[float]:
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self) -> str:
        return str(tuple(self))

    def __bytes__(self) -> bytes:
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other) -> bool:
        return tuple(self) == tuple(other)

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)

    def __bool__(self) -> bool:
        return bool(abs(self))

    def angle(self) -> float:
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec='') -> str:
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets) -> Vector2d:
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V5_COMPLEX[]
    def __abs__(self) -> float:  # <1>
        return math.hypot(self.x, self.y)

    def __complex__(self) -> complex:  # <2>
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum: SupportsComplex) -> Vector2d:  # <3>
        c = complex(datum)  # <4>
        return cls(c.real, c.imag)
# end::VECTOR2D_V5_COMPLEX[]


## Designing a Static Protocol

Recall the `Tombola` ABC which specifies two methods: `pick` and `load`.

Let's define a static protocol for `pick`.

Example 13-18. `randompick.py`: definition of `RandomPicker`

In [84]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...


Example 13-19. `randompick_test.py`: `RandomPicker` in use

In [89]:
from typing import Iterable, Any
import random
from typing import TypeVar

T = TypeVar("T")


class SimplePicker:  # <2>
    def __init__(self, items: Iterable[T]) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> T:  # <3>
        return self._items.pop()


In [90]:
popper: RandomPicker = SimplePicker([1])
isinstance(popper, RandomPicker)

True

In [93]:
from typing import TYPE_CHECKING


items = [1, 2]
popper = SimplePicker(items)
item = popper.pick()
assert item in items
if TYPE_CHECKING:
    reveal_type(item)  # generates a note in the mypy output
isinstance(item, int)

NameError: name 'reveal_type' is not defined

In [92]:
!mypy ./typing/randompick_test.py

typing/randompick_test.py:24: note: Revealed type is "Any"
Success: no issues found in 1 source file


## Best Practices for Protocol Design

Naming conventions for protocols:
- use plain names for protocols that represent a clear concept (e.g. `Iterator`, `Container`)
- Use `SupportsX` for protocols that provide callable methods (e.g. `SupportsInt`, `SupportsRead`, `SupportsReadSeek`)
- Use `HasX` for protocols that have readable and/or writable attributes or getter/setter methods (e.g. `HasItems`,
`HasFileno`)

## Extending a Protocol

When practice reveals that a protocol with more methods is useful, instead of adding methods to the original
protocol, it's better to derive a new protocol from it.

Example 13-20. `randompickload.py`: extending `RandomPicker`

In [94]:
from typing import Protocol, runtime_checkable

@runtime_checkable  # <1>
class LoadableRandomPicker(RandomPicker, Protocol):  # <2>
    def load(self, Iterable) -> None: ...  # <3>


## The numbers ABCs and Numeric Protocols



In [95]:

import numpy as np

cd = np.cdouble(3 + 4j)
cd

(3+4j)

In [96]:
float(cd)

/var/folders/hd/cjhmmycx76qbkj00_2prr05r0000gr/T/ipykernel_96343/1425064472.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  float(cd)


3.0

In [98]:
from typing import SupportsComplex

sample = [
    1+0j,  # yes
    np.complex64(1+0j), # yes - might have issue with 32 bit or 64 bit
    1.0,  # yes
    np.float16(1.0),  # probably?
    1,  # yes
    np.uint8(1), # no
]

In [99]:
[isinstance(x, SupportsComplex) for x in sample]

[False, True, False, False, False, False]

In [100]:
[complex(x) for x in sample]

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]

The `numbers` ABCs are fine for runtime type checking, but unsuitable for static type checking.

The numeric static protocols `SupportsComplex`, `SupportsFloat`, etc. work well for static typing, but are unreliable
 for runtime type checking when complex numbers are involved.

## Chapter Summary

We consrasted dynamic and static protocols, which respectively support duck typing and static duck typing. A class
supports a protocol simply by implementing the necessary methods.

The next section was "Programming Ducks". This ended with hints for defensive programming.

We introduced goose typing and saw how to subclass existing ABCs. We created an ABC from scratch.

The last major section was "Static Protocols", where we resumed coverage of static duck typing.

We have 4 complementary ways of programming with interfaces in modern Python, each with different advantages and
drawbacks. Rejecting any of these approaches will make your work as a Python programmer harder than it needs to be.
